# DLC Live PyTorch Demo

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dlclive import DLCLive
import cv2
import numpy as np
from pathlib import Path
import time

### Snapshot to ONNX model 

In [ ]:
# In case you do not have a .onnx model exported, use this cell to export your DLC3.0 snapshot

from deeplabcut.pose_estimation_pytorch.config import read_config_as_dict
from deeplabcut.pose_estimation_pytorch.models import PoseModel
import torch
import onnxruntime as ort

device = "cuda" if torch.cuda.is_available() else "cpu"

#Dikra
# root = Path("/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy")
# model_cfg = read_config_as_dict(root / "pytorch_config.yaml")
# weights_path = root / "snapshot-200.pt"

#Anna
root = Path("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train")
model_cfg = read_config_as_dict(root / "pytorch_config.yaml")
weights_path = root / "snapshot-263.pt"

model = PoseModel.build(model_cfg["model"])
weights = torch.load(weights_path, map_location=device)
model.load_state_dict(weights["model"])

dummy_input = torch.zeros((1, 3, 224, 224))

torch.onnx.export(
    model,
    dummy_input,
    "/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train/resnet.onnx",
    verbose=False,
    input_names=["input"],
    dynamic_axes={"input": {0: "batch_size", 2: "height", 3: "width"}},
)

In [ ]:
# Load test frame
img = cv2.imread("/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/img008.png")

### DLC Live with ONNX exported DLC 3.0 model

In [ ]:
# Dikra
onnx_dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    model_type="onnx",
    device="cuda",
    display=True,
)

# Anna
# onnx_dlc_live = DLCLive(pytorch_cfg="/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train", processor=dlc_proc, snapshot='/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train/snapshot-263.pt')
# onnx_dlc_live = DLCLive("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/DLC_dev-single-animal_resnet_50_iteration-1_shuffle-1", processor=dlc_proc)
# img = cv2.imread("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/img049.png")

onnx_pose = onnx_dlc_live.init_inference(frame=img)
onnx_pose

![Screenshot from 2024-08-20 14-29-53.png](./docs/assets/Screenshot%20from%202024-08-20%2014-36-00.png)

In [ ]:
onnx_pose = onnx_dlc_live.get_pose(frame=img)
onnx_pose

In [ ]:
onnx_dlc_live.display.destroy()

### DLC Live with snaptshot of DLC 3.0 model (.pt)

In [ ]:
# Dikra
pytorch_dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    snapshot="snapshot-200.pt",
    device="cuda",
    model_type="pytorch",
    display=True,
)

# Anna
# pytorch_dlc_live = DLCLive(pytorch_cfg="/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train", processor=dlc_proc, snapshot='/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train/snapshot-263.pt')
# pytorch_dlc_live = DLCLive("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/DLC_dev-single-animal_resnet_50_iteration-1_shuffle-1", processor=dlc_proc)
# img = cv2.imread("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/img049.png")

pytorch_pose = pytorch_dlc_live.init_inference(frame=img)
pytorch_pose

In [ ]:
pytorch_dlc_live.display.destroy()

![PyTorch model inference](./docs/assets/Screenshot%20from%202024-08-20%2014-29-53.png)

### Which is faster?

In [ ]:
import glob
import os

root = "/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy"
test_images = glob.glob(os.path.normpath(root + "/*.png"))


def mean_time_inference(dlc_live, images):
    times = []
    for i, img_p in enumerate(images):
        img = cv2.imread(img_p)

        if i == 0:
            start = time.time()
            dlc_live.init_inference(img)
            end = time.time()
        else:
            start = time.time()
            dlc_live.get_pose(img)
            end = time.time()
        times.append(end - start)
    print(times)

    return np.mean(times), times

In [ ]:
dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    device="cuda",
    model_type="onnx",
    display=True,
)

mean_time = mean_time_inference(dlc_live, test_images)
print(
    f"TOTAL Inference of ONNX model took on average {mean_time} seconds for {len(test_images)} images"
)

In [ ]:
dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    snapshot="snapshot-200.pt",
    device="cuda",
    model_type="pytorch"
)
root = "/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy"
test_images = glob.glob(os.path.normpath(root + "/*.png"))

mean_time = mean_time_inference(dlc_live, test_images) 
print(f"Inference of PyTorch model took on average {mean_time} seconds for {len(test_images)} images")

# Benchmarking

Currently the benchmark_pytorch.py script serves to provide a function for analyzing a preexisting video to test PyTorch for running video inference in DLC-Live. Code for running video inference on a live video feed is WIP.

For true benchmarking purposes, we aim to add feature for recording the time it takes to analyze each frame / how many frames can be analyzed per second. Discuss what measure to use and consult the DLC Live paper

In [1]:
%load_ext autoreload
%autoreload 2

# Import the analyze_video function from the file where it's defined
from dlclive.benchmark_pytorch import analyze_video

Loading DLC 3.0.0rc2...


/Users/annastuckert/anaconda3/envs/deeplabcut3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/annastuckert/anaconda3/envs/deeplabcut3/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [19]:
# New version with DLCLive object included in the code



# Define the paths
video_path = '/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/1_20cms_0degUP_first_03s.avi'
model_path = '/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train'

import cProfile
import io
import pstats

pr = cProfile.Profile()
pr.enable()

# Call the analyze_video function with the appropriate arguments
poses = analyze_video(
    video_path=video_path,
    model_path=model_path,
    model_type="onnx",
    device="cuda",      
    display=True,      
    save_poses=True,
    resize= 0.5,
    #cropping= [50, 250, 100, 450], manually set the cropping to specific pixels
    dynamic=(True, 0.5, 10), #True = we want to apply dynamic cropping, 0.5 = the threshold for accepting a KP as detected, 10 = the margin to expand the calculatted cropping window by so it is not too narrow
    save_dir='output_directory',  
    draw_keypoint_names=True      
)

# #'poses' will contain the list of poses detected

# # Create a stream to capture the profiler's output
# s = io.StringIO()
# sortby = 'cumulative'
# ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
# ps.print_stats()

# # Print the profiling output
# print(s.getvalue())

Exception ignored When destroying _lsprof profiler:
Traceback (most recent call last):
  File "/var/folders/3y/jmv30wb16ss901m1yrdysjh80000gq/T/ipykernel_50549/776358094.py", line 14, in <module>
RuntimeError: Cannot install a profile function while another profile function is being installed


ONNX inference took 0.08013701438903809 sec
ONNX inference took 0.023272991180419922 sec
Frame 0 processing time: 0.2047 seconds
ONNX inference took 0.08839082717895508 sec
ONNX inference took 0.021580934524536133 sec
Frame 1 processing time: 0.1312 seconds
ONNX inference took 0.10285806655883789 sec
ONNX inference took 0.03156018257141113 sec
Frame 2 processing time: 0.1646 seconds
ONNX inference took 0.21061491966247559 sec
ONNX inference took 0.03587079048156738 sec
Frame 3 processing time: 0.2813 seconds
ONNX inference took 0.15520811080932617 sec
ONNX inference took 0.033479928970336914 sec
Frame 4 processing time: 0.2313 seconds
ONNX inference took 0.14543604850769043 sec
ONNX inference took 0.024671077728271484 sec
Frame 5 processing time: 0.2139 seconds
ONNX inference took 0.1022939682006836 sec
ONNX inference took 0.02432107925415039 sec
Frame 6 processing time: 0.1651 seconds
ONNX inference took 0.10728287696838379 sec
ONNX inference took 0.028424978256225586 sec
Frame 7 proc

In [ ]:
# test download of benchmarking dataset 
#OBS link it not working, waiting for updated link to benchmarking dataset

# from dlclive.benchmark_pytorch import download_benchmarking_data

# download_benchmarking_data()



In [ ]:
from dlclive.benchmark_pytorch import get_system_info


sys_info = get_system_info()

In [ ]:
print(sys_info)